In [7]:
import tensorflow as tf

In [8]:
cifar10 = tf.keras.datasets.cifar10
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
train_images = train_images / 255.0
test_images = test_images / 255.0

val_images = train_images[-10000:]
val_labels = train_labels[-10000:]
train_images = train_images[:-10000]
train_labels = train_labels[:-10000]

print(train_labels.shape)
print(val_labels.shape)

170508288/170498071 [==============================] - 2s 0us/step
(40000, 1)
(10000, 1)


In [9]:
# Squeeze and Excite layer
def fire_module(x, s_1x1, e_1x1, e_3x3):
  x = tf.keras.layers.Conv2D(s_1x1, 1, padding="SAME", activation="relu")(x)
  x_e_1x1 = tf.keras.layers.Conv2D(e_1x1, 1, padding="SAME", activation="relu")(x)
  x_e_3x3 = tf.keras.layers.Conv2D(e_3x3, 3, padding="SAME", activation="relu")(x)

  x = tf.concat([x_e_1x1, x_e_3x3], 3)

  return x

<img src="https://pytorch.org/assets/images/squeezenet.png" />

In [10]:
resize_layer = tf.keras.layers.Lambda( 
    lambda image: tf.image.resize( 
        image, 
        (224, 224), 
        method = tf.image.ResizeMethod.BICUBIC,
        preserve_aspect_ratio = True
    )
)

inp = tf.keras.layers.Input(shape=(32, 32, 3))
x = resize_layer(inp)
x = tf.keras.layers.Conv2D(96, 7, strides=2)(x)
x = tf.keras.layers.MaxPool2D(3, 2)(x)

x = fire_module(x, 16, 64, 64)
x = fire_module(x, 16, 64, 64)
x = fire_module(x, 32, 128, 128)
x = tf.keras.layers.MaxPool2D(3, 2)(x)

x = fire_module(x, 32, 128, 128)
x = fire_module(x, 48, 192, 192)
x = fire_module(x, 48, 192, 192)
x = fire_module(x, 64, 256, 256)
x = tf.keras.layers.MaxPool2D(3, 2)(x)

x = fire_module(x, 64, 256, 256)
x = tf.keras.layers.Conv2D(1000, 1)(x)
x = tf.keras.layers.AveragePooling2D()(x)

x = tf.keras.layers.Flatten()(x)
        
x = tf.keras.layers.Dense(4096)(x)
x = tf.keras.layers.Dense(4096)(x)
out = tf.keras.layers.Dense(10, activation="softmax")(x)

model = tf.keras.Model(inputs=inp, outputs=out)


In [11]:
ts_inp = tf.random.normal((1, 32, 32, 3))
ts_out = model(ts_inp)
print(model.summary())

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 lambda_1 (Lambda)              (None, 224, 224, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv2d_26 (Conv2D)             (None, 109, 109, 96  14208       ['lambda_1[0][0]']               
                                )                                                                 
                                                                                                  
 max_pooling2d_3 (MaxPooling2D)  (None, 54, 54, 96)  0           ['conv2d_26[0][0]']        

In [12]:
model.compile(optimizer='sgd', 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

In [ ]:
history = model.fit(
  x=train_images,
  y=train_labels,
  batch_size=32,
  epochs=10,
  validation_data=(val_images, val_labels)
)

Epoch 1/10


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


 500/1250 [===========>..................] - ETA: 1:18:59 - loss: 2.3032 - sparse_categorical_accuracy: 0.0995